# Data Collection and Cleaning
The main data set that I chose for this project was NBA Season Statistics that was found on [Kaggle](https://www.kaggle.com/drgilermo/nba-players-stats#Seasons_Stats.csv). This data set includes basic stats such as points, field goal percentage, fouls, etc. But it also contains advanced statistics such player efficiency rating, win shares, offensive rebound percentage, etc. I will explain what some of these statistics actually attempt to show in other parts of this project

**Important Note:** This data set is organized by year. But because one season in the NBA spans over two different calendar years, for this data, 2017 corresponds to the 2016-2017 season.

In [0]:
import pandas as pd
import numpy as np
# Data set was found on Kaggle
df_stats = pd.read_csv("/content/drive/My Drive/Seasons_Stats.csv")

## Add an experience column to the DataFrame

In [0]:
"""
Function that adds how many years a player has been in the NBA
Args: 
  -df: type pd.DataFrame
Returns:
  type pd.DataFrame with a new "Experience" column with the number of years a 
  player has been in NBA

NOTE: -Must pass a DataFrame that is sorted by Player and then by Year
      -Must keep track if a player shows up multiple times in the same year 
       (because of trades)
"""
def years_played(df):
  player_year_dict = {}
  name_dict = {}
  for index, row in df.iterrows():
    player = row["Player"]
    year = row["Year"]
    
    # Check if the player has been seen
    if (player not in name_dict):
      name_dict[player] = 1
      df.at[index, "Experience"] = 1
      player_year_dict = {}
      player_year_dict[player] = [year]
     
    # Check if the player has not been on multiple teams
    elif (year not in player_year_dict[player]):
      name_dict[player] += 1
      player_year_dict[player].append(year)
      df.at[index, "Experience"] = name_dict[player]

    # Player has been on multiple teams. Don't increment their experience
    else:
      df.at[index, "Experience"] = name_dict[player]
  return df

## Filter Data
Filter the data to have somewhat current players and add the Experience column using the function from above

In [8]:
# Remove the empty rows that are used as space holders in between years
df_stats = df_stats[~df_stats["Player"].isnull()]

#Sort DF by the Player and then Year
df_stats = df_stats.sort_values(["Player", "Year"], ascending=[True, True])

# Add the Experience column to the current players DF
df_experience = years_played(df_stats)

# Re-sort the DF by Year and Player
df_experience = df_experience.sort_values(["Year", "Player"], ascending=[True, True])

# Subset DF to be of somewhat current players
df_experience = df_experience[df_experience["Year"] >= 1980]
df_experience.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Experience
5881,5881,1980.0,Abdul Jeelani,SF,25.0,POR,77.0,NaN,1286.0,18.7,0.563,0.011,0.361,9.6,13.5,11.5,11.6,1.5,1.8,15.2,24.6,NaN,2.3,1.5,3.8,0.141,NaN,1.0,-0.8,0.2,0.7,288.0,565.0,0.510,0.0,6.0,0.000,288.0,559.0,0.515,0.510,161.0,204.0,0.789,114.0,156.0,270.0,95.0,40.0,40.0,117.0,155.0,737.0,1.0
5807,5807,1980.0,Adrian Dantley*,SF,23.0,UTA,68.0,NaN,2674.0,24.3,0.635,0.002,0.415,8.3,14.0,11.3,12.3,1.8,0.3,13.5,27.8,NaN,9.5,1.1,10.5,0.189,NaN,5.6,-1.9,3.6,3.8,730.0,1267.0,0.576,0.0,2.0,0.000,730.0,1265.0,0.577,0.576,443.0,526.0,0.842,183.0,333.0,516.0,191.0,96.0,14.0,233.0,211.0,1903.0,4.0
6026,6026,1980.0,Al Skinner,SG,27.0,PHI,2.0,NaN,10.0,-0.6,0.500,0.000,0.000,0.0,0.0,0.0,25.4,0.0,0.0,50.0,16.0,NaN,0.0,0.0,0.0,-0.152,NaN,-11.8,-8.2,-20.0,0.0,1.0,2.0,0.500,0.0,0.0,NaN,1.0,2.0,0.500,0.500,0.0,0.0,NaN,0.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,2.0,4.0
5828,5828,1980.0,Alex English*,SF,26.0,TOT,78.0,NaN,2401.0,18.0,0.536,0.005,0.239,11.3,14.9,13.1,13.9,1.4,1.4,14.8,23.4,NaN,3.3,2.0,5.3,0.105,NaN,1.4,-0.5,1.0,1.8,553.0,1113.0,0.497,2.0,6.0,0.333,551.0,1107.0,0.498,0.498,210.0,266.0,0.789,269.0,336.0,605.0,224.0,73.0,62.0,214.0,206.0,1318.0,4.0
5829,5829,1980.0,Alex English*,SF,26.0,IND,54.0,NaN,1526.0,17.7,0.539,0.004,0.204,11.0,14.9,12.9,13.5,1.4,1.2,14.2,22.2,NaN,2.4,1.3,3.6,0.114,NaN,1.3,-0.7,0.6,1.0,346.0,686.0,0.504,0.0,3.0,0.000,346.0,683.0,0.507,0.504,114.0,140.0,0.814,167.0,213.0,380.0,142.0,45.0,33.0,124.0,128.0,806.0,4.0


In [6]:
# Subset DF to be of somewhat current players
df_current = df_stats[df_stats["Year"] >= 1980]


df_current = df_current[~df_current["Player"].isnull()]

#Sort DF by the Player and then Year
df_current = df_current.sort_values(["Player", "Year"], ascending=[True, True])

# Add the Experience column to the current players DF
df_experience = years_played(df_current)

# Re-sort the DF by Year and Player
df_experience = df_experience.sort_values(["Year", "Player"], ascending=[True, True])
df_experience.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Experience
5881,5881,1980.0,Abdul Jeelani,SF,25.0,POR,77.0,NaN,1286.0,18.7,0.563,0.011,0.361,9.6,13.5,11.5,11.6,1.5,1.8,15.2,24.6,NaN,2.3,1.5,3.8,0.141,NaN,1.0,-0.8,0.2,0.7,288.0,565.0,0.510,0.0,6.0,0.000,288.0,559.0,0.515,0.510,161.0,204.0,0.789,114.0,156.0,270.0,95.0,40.0,40.0,117.0,155.0,737.0,1.0
5807,5807,1980.0,Adrian Dantley*,SF,23.0,UTA,68.0,NaN,2674.0,24.3,0.635,0.002,0.415,8.3,14.0,11.3,12.3,1.8,0.3,13.5,27.8,NaN,9.5,1.1,10.5,0.189,NaN,5.6,-1.9,3.6,3.8,730.0,1267.0,0.576,0.0,2.0,0.000,730.0,1265.0,0.577,0.576,443.0,526.0,0.842,183.0,333.0,516.0,191.0,96.0,14.0,233.0,211.0,1903.0,1.0
6026,6026,1980.0,Al Skinner,SG,27.0,PHI,2.0,NaN,10.0,-0.6,0.500,0.000,0.000,0.0,0.0,0.0,25.4,0.0,0.0,50.0,16.0,NaN,0.0,0.0,0.0,-0.152,NaN,-11.8,-8.2,-20.0,0.0,1.0,2.0,0.500,0.0,0.0,NaN,1.0,2.0,0.500,0.500,0.0,0.0,NaN,0.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,2.0,1.0
5828,5828,1980.0,Alex English*,SF,26.0,TOT,78.0,NaN,2401.0,18.0,0.536,0.005,0.239,11.3,14.9,13.1,13.9,1.4,1.4,14.8,23.4,NaN,3.3,2.0,5.3,0.105,NaN,1.4,-0.5,1.0,1.8,553.0,1113.0,0.497,2.0,6.0,0.333,551.0,1107.0,0.498,0.498,210.0,266.0,0.789,269.0,336.0,605.0,224.0,73.0,62.0,214.0,206.0,1318.0,1.0
5829,5829,1980.0,Alex English*,SF,26.0,IND,54.0,NaN,1526.0,17.7,0.539,0.004,0.204,11.0,14.9,12.9,13.5,1.4,1.2,14.2,22.2,NaN,2.4,1.3,3.6,0.114,NaN,1.3,-0.7,0.6,1.0,346.0,686.0,0.504,0.0,3.0,0.000,346.0,683.0,0.507,0.504,114.0,140.0,0.814,167.0,213.0,380.0,142.0,45.0,33.0,124.0,128.0,806.0,1.0


## Web Scraping for Salaries
I want to analyze players salaries along with their significant statistics for a given season. So using BeautifulSoup, I webscraped each player salaries from their respective years from [hoopshype.com](https://hoopshype.com).

In [0]:
import requests
import time
from bs4 import BeautifulSoup

In [0]:
df_stats2010 = df_experience[df_experience["Year"] >= 2010]

In [0]:
"""
Function that creates a data frame of all salaries for every player in given 
year range

Args: start - start year of when to start the scrapping (int ie. 2013)
      end - end of year of when to stop scrapping (int ie. 2015)

Returns: DataFrame of all the salaries for each player in the given year range

NOTE: 2013 corresponds to 2012/2013 season
"""
def web_scrape_salaries(start, end):
  df_final = pd.DataFrame(columns=["Player", "Salary", "Year"])
  for season in range(start, end + 1):
    # Go to the website for the current season
    response = (requests.get("https://hoopshype.com/salaries/players/%d-%d/" % 
                             (season - 1, season))
    )
    
    soup = BeautifulSoup(response.content, "html.parser")
    tables = soup.find_all("table")
    table = tables[0]

    # Get the name and salary for each player
    rows_list = []
    for player in table.find_all("tr")[1:]:
      cells = player.find_all("td")
  
      row = {}
      name = cells[1].find("a").text.strip()
      name = name.replace("[^\w\s]", "")
      salary = int(cells[2].get("data-value"))
      row.update({"Player": name, "Salary": salary, "Year": float(season)})
      rows_list.append(row)

    # Create the data frame for the current season
    df = pd.DataFrame(rows_list, columns=["Player", "Salary", "Year"])

    # Concatenate the current data frame with the final data frame
    df_final = pd.concat([df_final, df], ignore_index=True)

  return df_final

In [0]:
# For machine learning model use from 2010-2017
df_web_salaries = web_scrape_salaries(2010, 2017)

In [13]:
df_sample = df_stats2010.merge(df_web_salaries, on=["Player", "Year"], how="inner")
df_sample.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Experience,Salary
0,19996,2010.0,Aaron Brooks,PG,25.0,HOU,82.0,82.0,2919.0,16.0,0.549,0.394,0.224,2.0,6.6,4.2,25.6,1.2,0.3,13.7,25.7,NaN,3.9,1.6,5.5,0.091,NaN,2.7,-2.5,0.2,1.6,575.0,1331.0,0.432,209.0,525.0,0.398,366.0,806.0,0.454,0.511,245.0,298.0,0.822,54.0,161.0,215.0,434.0,69.0,14.0,232.0,199.0,1604.0,3.0,1118520
1,20120,2010.0,Aaron Gray,C,25.0,TOT,32.0,0.0,311.0,16.6,0.551,0.000,0.341,16.7,23.1,19.8,10.6,1.5,2.7,16.0,16.4,NaN,0.5,0.4,0.9,0.142,NaN,-0.6,1.3,0.7,0.2,42.0,82.0,0.512,0.0,0.0,NaN,42.0,82.0,0.512,0.512,20.0,28.0,0.714,45.0,61.0,106.0,22.0,9.0,11.0,18.0,54.0,104.0,3.0,1000497
2,20121,2010.0,Aaron Gray,C,25.0,CHI,8.0,0.0,50.0,7.0,0.374,0.000,0.333,11.3,24.1,17.8,6.5,0.0,0.0,4.0,22.5,NaN,-0.1,0.1,0.0,-0.013,NaN,-6.6,-3.7,-10.2,-0.1,8.0,21.0,0.381,0.0,0.0,NaN,8.0,21.0,0.381,0.381,2.0,7.0,0.286,5.0,11.0,16.0,2.0,0.0,0.0,1.0,10.0,18.0,3.0,1000497
3,20122,2010.0,Aaron Gray,C,25.0,NOH,24.0,0.0,261.0,18.4,0.612,0.000,0.344,17.7,22.9,20.2,11.4,1.8,3.2,19.5,15.3,NaN,0.6,0.3,0.9,0.172,NaN,0.5,2.2,2.8,0.3,34.0,61.0,0.557,0.0,0.0,NaN,34.0,61.0,0.557,0.557,18.0,21.0,0.857,40.0,50.0,90.0,20.0,9.0,11.0,17.0,44.0,86.0,3.0,1000497
4,20248,2010.0,Acie Law,PG,25.0,TOT,26.0,1.0,234.0,16.0,0.585,0.213,0.653,1.4,6.6,4.1,17.5,2.3,0.4,15.0,21.7,NaN,0.3,0.2,0.6,0.115,NaN,-0.7,-2.5,-3.2,-0.1,35.0,75.0,0.467,5.0,16.0,0.313,30.0,59.0,0.508,0.500,38.0,49.0,0.776,3.0,14.0,17.0,26.0,11.0,1.0,17.0,16.0,113.0,3.0,2216160


In [0]:
# Create per game statistics
stats = ["FGA", "TRB", "AST", "STL", "BLK", "AST", "TOV", "FGA", "FG", "3PA",
         "3P", "MP", "PTS"]

for stat in stats:
  df_sample["%s/G" % stat] = round(df_sample[stat] / df_sample["G"], 3)
df_sample["Salary"] = df_sample["Salary"].astype(int)

In [0]:
# Save the file to be used in other directories
df_sample.to_csv("df_sample.csv", encoding='utf-8', index=False)